In [1]:
import numpy as np
import torch
import datasets
import pandas as pd
# import editdistance
import transformers
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from accelerate import Accelerator
from sklearn.model_selection import StratifiedShuffleSplit
import os
from tqdm import tqdm
import re




===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/user/ksharma/miniconda3/envs/ks_tars_thesis did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib64')}
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: No libcudart.so found! Install CUDA or the cudatoolkit package (anaconda)!
  warn(msg)


In [247]:
# results_gpt3_P1_df.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/GPT3_SS/results_gpt3_P1_df.csv")
# results_gpt3_P2_df.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/GPT3_SS/results_gpt3_P2_df.csv")
# results_chatGPT_df.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/GPT3_SS/results_chatGPT_df.csv")

# results_chatGPT_df = pd.read_csv('/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/GPT3_SS/results_chatGPT_df.csv')

In [6]:
results_chatGPT_df['Problem type'].value_counts()

Problem type
OWA     15
HOWA     8
MC       5
MWA      4
NONE     2
MDC      1
Name: count, dtype: int64

In [4]:
results_chatGPT_df = pd.read_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/chatGPT_eval.csv")
results_chatGPT_df

,input,generated_text,expected_text,Wrong chatGPT,Problem type
0,Technische Kenntnisse\n<SoftSkill>Gespür für K...,Technische Kenntnisse\n<SoftSkill>Gespür für K...,Technische Kenntnisse\n<SoftSkill>Gespür für K...,True,HOWA
1,Anforderungsprofil\nDu bist ein/e <SoftSkill>m...,Anforderungsprofil\nDu bist ein/e <SoftSkill>m...,Anforderungsprofil\nDu bist ein/e <SoftSkill>m...,False,HOWA
2,"• <SoftSkill_C>freundliches</SoftSkill_C>, <S...","<SoftSkill>Freundliches Auftreten</SoftSkill>,...","<SoftSkill>Freundliches Auftreten</SoftSkill>,...",True,MC
3,Durch Ihre <SoftSkill_C>methodische</SoftSkill...,Durch Ihre <SoftSkill>methodische Arbeitsweise...,Durch Ihre <SoftSkill>methodische Arbeitsweise...,False,MC
4,Für unsere Abteilung Pâtisserie\nin der Delici...,Für unsere Abteilung Pâtisserie\nin der Delici...,Für unsere Abteilung Pâtisserie\nin der Delici...,True,OWA
5,<SoftSkill>In Zusammenhängen denken</SoftSkill...,<SoftSkill>In Zusammenhängen denken</SoftSkill...,<SoftSkill>In Zusammenhängen denken</SoftSkill...,True,NONE
6,Anforderungsprofil\nWir suchen für alle unsere...,Anforderungsprofil\nWir suchen für alle unsere...,Anforderungsprofil\nWir suchen für alle unsere...,True,HOWA
7,"<SoftSkill>Organisationstalent</SoftSkill>, <S...","<SoftSkill>Organisationstalent</SoftSkill>, <S...","<SoftSkill>Organisationstalent</SoftSkill>, <S...",False,MWA
8,Sie verfügen über <SoftSkill>unternehmerisches...,Sie verfügen über <SoftSkill>unternehmerisches...,Sie verfügen über <SoftSkill>unternehmerisches...,False,OWA
9,"Sie sind <SoftSkill>engagiert</SoftSkill>, <So...","Sie sind <SoftSkill>engagiert</SoftSkill>, <So...","Sie sind <SoftSkill>engagiert</SoftSkill>, <So...",False,HOWA


In [5]:
# results_chatGPT_df = results_chatGPT_df.drop(columns='USAGE FLAG', axis = 0)
# results_chatGPT_df = results_chatGPT_df.drop(columns='wrong GPT3', axis = 0)
results_chatGPT_df['expected_skills_all'] = results_chatGPT_df['expected_text'].str.findall(r'<SoftSkill>(.*?)<\/SoftSkill')
results_chatGPT_df['generated_skills_all'] = results_chatGPT_df['generated_text'].str.findall(r'(?:SoftSkill>|SoftSkill_C>)(.*?)(?:</SoftSkill>|</SoftSkill_C>)')
results_chatGPT_df.head(4)

,input,generated_text,expected_text,Wrong chatGPT,Problem type,expected_skills_all,generated_skills_all
0,Technische Kenntnisse\n<SoftSkill>Gespür für K...,Technische Kenntnisse\n<SoftSkill>Gespür für K...,Technische Kenntnisse\n<SoftSkill>Gespür für K...,True,HOWA,"[Gespür für Kundenzufriedenheit, Führungsfähig...","[Gespür für Kundenzufriedenheit, Führungsfähig..."
1,Anforderungsprofil\nDu bist ein/e <SoftSkill>m...,Anforderungsprofil\nDu bist ein/e <SoftSkill>m...,Anforderungsprofil\nDu bist ein/e <SoftSkill>m...,False,HOWA,"[motivierte/r Jugendliche/r, hoher Lernbereits...","[motivierte/r Jugendliche/r, hoher Lernbereits..."
2,"• <SoftSkill_C>freundliches</SoftSkill_C>, <S...","<SoftSkill>Freundliches Auftreten</SoftSkill>,...","<SoftSkill>Freundliches Auftreten</SoftSkill>,...",True,MC,"[Freundliches Auftreten, sauberes Auftreten, m...","[Freundliches Auftreten, sauberes Auftreten, m..."
3,Durch Ihre <SoftSkill_C>methodische</SoftSkill...,Durch Ihre <SoftSkill>methodische Arbeitsweise...,Durch Ihre <SoftSkill>methodische Arbeitsweise...,False,MC,"[methodische Arbeitsweise, strukturierte Arbei...","[methodische Arbeitsweise, strukturierte Arbei..."


In [250]:
# results_chatGPT_df[results_chatGPT_df['wrong GPT3'] == 0]

## GET TEST DATA 

In [7]:

import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge


# TESTING SAVED MODELS

In [8]:
import pandas as pd
import re
pd.set_option("display.max_colwidth", None)

# results_chatGPT_df = pd.read_csv("/home/user/ksharma/ks_thesis/ma-thesis/bloom_peft_lora/bloom_7bn_results_ORIG_15epochs.csv")
# results_chatGPT_df = results_chatGPT_df[results_chatGPT_df['valid'] == 1]
# results_chatGPT_df = results_chatGPT_df.drop(columns='Unnamed: 0', axis = 0)
# results_chatGPT_df = results_chatGPT_df.drop(columns='expected_skills', axis = 0)
# results_chatGPT_df = results_chatGPT_df.drop(columns='generated_skills', axis = 0)

# Function to extract skills between "<SoftSkill_C>" and "</SoftSkill_C>"
def extract_skills_c(text):
    return re.findall(r'<SoftSkill_C>(.*?)<\/SoftSkill_C', text)

# Function to extract skills between "<SoftSkill>"
def extract_skills(text):
    return re.findall(r'<SoftSkill>(.*?)<\/SoftSkill', text)

# Apply the functions to 'input', 'expected_text', and 'generated_text' columns
results_chatGPT_df['input_skills_C'] = results_chatGPT_df['input'].apply(extract_skills_c)
# df['input_skills'] = df['input'].apply(extract_skills)
results_chatGPT_df['input_skills_all'] = results_chatGPT_df['input'].str.findall(r'(?:<SoftSkill>|<SoftSkill_C>)(.*?)(?:</SoftSkill|</SoftSkill_C)')
# results_chatGPT_df['expected_skills_all'] = results_chatGPT_df['expected_text'].str.findall(r'<SoftSkill>(.*?)<\/SoftSkill')
# results_chatGPT_df['generated_skills_all'] = results_chatGPT_df['generated_text'].str.findall(r'SoftSkill>(.*?)/SoftSkill>')
# results_chatGPT_df


In [253]:
# results_chatGPT_df[results_chatGPT_df['orig_idx'] == 1910]

In [9]:
# Function to find the corresponding skills in 'expected_text' and 'generated_text'
def find_C_expected_skills(row):
    corresponding_skills = []
    for skill_c in row['input_skills_C']:
        if skill_c in row['input_skills_all']:
            idx = row['input_skills_all'].index(skill_c)
            corresponding_skills.append(row['expected_skills_all'][idx])
            # corresponding_skills.append(row['generated_skills_new'][idx])
    return corresponding_skills

# def find_C_generated_skills(row):
#     corresponding_skills = []
#     for skill_c, generated_skill in zip(row['input_skills_C'], row['generated_skills_all']):
#         if skill_c in row['input_skills_all']:
#             idx = row['input_skills_all'].index(skill_c)
#             corresponding_skills.append(generated_skill)
#     return corresponding_skills

def find_C_generated_skills(row):
    corresponding_skills = []
    # print(f"input_skills_C = {row['input_skills_C']}")
    # print(f"end=input_skills_all = {row['input_skills_all']}")
    # print(f"generated_skills_all = {row['generated_skills_all']}")
    if len(row['input_skills_all']) == len(row['generated_skills_all']):
        for skill_c in row['input_skills_C']:
            if skill_c in row['input_skills_all']:
                idx = row['input_skills_all'].index(skill_c)
            corresponding_skills.append(row['generated_skills_all'][idx])
    return corresponding_skills

# Apply the function to create 'corresponding_skills' column
results_chatGPT_df['expected_skills_C'] = results_chatGPT_df.apply(find_C_expected_skills, axis=1)
results_chatGPT_df['generated_skills_C'] = results_chatGPT_df.apply(find_C_generated_skills, axis=1)


In [255]:
# corresponding_skills = []

# input_C = ['Fähigkeit', 'motivieren']
# input_all = ['Fähigkeit', 'ein Team zu begeistern', 'motivieren']
# indexes = []
# generated_all = ['Fähigkeit in ama', 'ein Team zu begeistern', 'die Fähigkeit, ein Team zu motivieren']
# for skill_c in input_C:
#     print(skill_c)
#     if skill_c in input_all:
#         idx = input_all.index(skill_c)
#     corresponding_skills.append(generated_all[idx])
# corresponding_skills

In [10]:
results_chatGPT_df['num_skills_all'] = results_chatGPT_df.apply(lambda row: len(row['expected_skills_all']) if isinstance(row['expected_skills_all'], list) else 0, axis=1)
results_chatGPT_df['num_skills_C'] = results_chatGPT_df.apply(lambda row: len(row['expected_skills_C']) if isinstance(row['expected_skills_C'], list) else 0, axis=1)

results_chatGPT_df['num_gen_skills_all'] = results_chatGPT_df.apply(lambda row: len(row['generated_skills_all']) if isinstance(row['generated_skills_all'], list) else 0, axis=1)
results_chatGPT_df['num_gen_skills_C'] = results_chatGPT_df.apply(lambda row: len(row['generated_skills_C']) if isinstance(row['generated_skills_C'], list) else 0, axis=1)


results_chatGPT_df.head(3)

,input,generated_text,expected_text,Wrong chatGPT,Problem type,expected_skills_all,generated_skills_all,input_skills_C,input_skills_all,expected_skills_C,generated_skills_C,num_skills_all,num_skills_C,num_gen_skills_all,num_gen_skills_C
0,Technische Kenntnisse\n<SoftSkill>Gespür für Kundenzufriedenheit</SoftSkill>\n<SoftSkill>Führungsfähigkeiten</SoftSkill>\nOrganisation der Dienstleistungen\n<SoftSkill_C>Analyse-</SoftSkill_C> und <SoftSkill>Beobachtungsfähigkeit bei Qualitätskontrollen</SoftSkill>\nVerständnis für Service,Technische Kenntnisse\n<SoftSkill>Gespür für Kundenzufriedenheit</SoftSkill>\n<SoftSkill>Führungsfähigkeiten</SoftSkill>\nOrganisation der Dienstleistungen\n<SoftSkill>Analysefähigkeit bei Qualitätskontrollen</SoftSkill> und <SoftSkill>Beobachtungsfähigkeit bei Qualitätskontrollen</SoftSkill>\nVerständnis für Service,Technische Kenntnisse\n<SoftSkill>Gespür für Kundenzufriedenheit</SoftSkill>\n<SoftSkill>Führungsfähigkeiten</SoftSkill>\nOrganisation der Dienstleistungen\n<SoftSkill>Analysefähigkeit bei Qualitätskontrollen</SoftSkill> und <SoftSkill>Beobachtungsfähigkeit bei Qualitätskontrollen</SoftSkill>\nVerständnis für Service,True,HOWA,"[Gespür für Kundenzufriedenheit, Führungsfähigkeiten, Analysefähigkeit bei Qualitätskontrollen, Beobachtungsfähigkeit bei Qualitätskontrollen]","[Gespür für Kundenzufriedenheit, Führungsfähigkeiten, Analysefähigkeit bei Qualitätskontrollen, Beobachtungsfähigkeit bei Qualitätskontrollen]",[Analyse-],"[Gespür für Kundenzufriedenheit, Führungsfähigkeiten, Analyse-, Beobachtungsfähigkeit bei Qualitätskontrollen]",[Analysefähigkeit bei Qualitätskontrollen],[Analysefähigkeit bei Qualitätskontrollen],4,1,4,1
1,"Anforderungsprofil\nDu bist ein/e <SoftSkill>motivierte/r Jugendliche/r</SoftSkill> mit <SoftSkill_C>hoher Lern-</SoftSkill_C> und <SoftSkill>Leistungsbereitschaft</SoftSkill> und legst grossen Wert auf <SoftSkill>Teamfähigkeit</SoftSkill>, <SoftSkill>Eigenverantwortung</SoftSkill>, <SoftSkill>Selbstständigkeit</SoftSkill> und <SoftSkill>Flexibilität</SoftSkill>.","Anforderungsprofil\nDu bist ein/e <SoftSkill>motivierte/r Jugendliche/r</SoftSkill> mit <SoftSkill>hoher Lernbereitschaft</SoftSkill> und <SoftSkill>hoher Leistungsbereitschaft</SoftSkill> und legst grossen Wert auf <SoftSkill>Teamfähigkeit</SoftSkill>, <SoftSkill>Eigenverantwortung</SoftSkill>, <SoftSkill>Selbstständigkeit</SoftSkill> und <SoftSkill>Flexibilität</SoftSkill>.","Anforderungsprofil\nDu bist ein/e <SoftSkill>motivierte/r Jugendliche/r</SoftSkill> mit <SoftSkill>hoher Lernbereitschaft</SoftSkill> und <SoftSkill>hoher Leistungsbereitschaft</SoftSkill> und legst grossen Wert auf <SoftSkill>Teamfähigkeit</SoftSkill>, <SoftSkill>Eigenverantwortung</SoftSkill>, <SoftSkill>Selbstständigkeit</SoftSkill> und <SoftSkill>Flexibilität</SoftSkill>.",False,HOWA,"[motivierte/r Jugendliche/r, hoher Lernbereitschaft, hoher Leistungsbereitschaft, Teamfähigkeit, Eigenverantwortung, Selbstständigkeit, Flexibilität]","[motivierte/r Jugendliche/r, hoher Lernbereitschaft, hoher Leistungsbereitschaft, Teamfähigkeit, Eigenverantwortung, Selbstständigkeit, Flexibilität]",[hoher Lern-],"[motivierte/r Jugendliche/r, hoher Lern-, Leistungsbereitschaft, Teamfähigkeit, Eigenverantwortung, Selbstständigkeit, Flexibilität]",[hoher Lernbereitschaft],[hoher Lernbereitschaft],7,1,7,1
2,"• <SoftSkill_C>freundliches</SoftSkill_C>, <SoftSkill_C>sauberes</SoftSkill_C> und <SoftSkill>motiviertes Auftreten</SoftSkill>","<SoftSkill>Freundliches Auftreten</SoftSkill>, <SoftSkill>sauberes Auftreten</SoftSkill> und <SoftSkill>motiviertes Auftreten</SoftSkill>","<SoftSkill>Freundliches Auftreten</SoftSkill>, <SoftSkill>sauberes Auftreten</SoftSkill> und <SoftSkill>motiviertes Auftreten</SoftSkill>",True,MC,"[Freundliches Auftreten, sauberes Auftreten, motiviertes Auftreten]","[Freundliches Auftreten, sauberes Auftreten, motiviertes Auftreten]","[freundliches, sauberes]","[freundliches, sauberes, motiviertes Auftreten]","[Freundliches Auftrete

In [257]:
# results_chatGPT_df.loc[results_chatGPT_df['num_skills_C'] == 0, 'input'] = results_chatGPT_df.loc[results_chatGPT_df['num_skills_C'] == 0, 'input'].apply(lambda x: x + '>')

In [11]:
results_chatGPT_df[results_chatGPT_df['num_skills_C']==0]

,input,generated_text,expected_text,Wrong chatGPT,Problem type,expected_skills_all,generated_skills_all,input_skills_C,input_skills_all,expected_skills_C,generated_skills_C,num_skills_all,num_skills_C,num_gen_skills_all,num_gen_skills_C


In [12]:

from collections import Counter

# Create a new column 'skills_match_score'
results_chatGPT_df['percent_skills_match_all'] = 0.0
results_chatGPT_df['percent_skills_match_C'] = 0.0

results_chatGPT_df['num_skills_match_all'] = 0.0
results_chatGPT_df['num_skills_match_C'] = 0.0

# Iterate over the rows of the DataFrame
for index, row in results_chatGPT_df.iterrows():
    expected_skills_all = set(row['expected_skills_all'])
    generated_skills_all = set(row['generated_skills_all'])
    
    expected_skills_C = set(row['expected_skills_C'])
    generated_skills_C = set(row['generated_skills_C'])

    count_gen_C = Counter(generated_skills_C)
    count_exp_C = Counter(expected_skills_C)

    # Find the common elements between list1 and list2
    common_elements_C = set(expected_skills_C).intersection(set(generated_skills_C))

    # Find the number of common elements, considering duplicates
    num_common_elements_C = sum(min(count_gen_C[element], count_exp_C[element]) for element in common_elements_C)

    count_gen_all = Counter(generated_skills_all)
    count_exp_all = Counter(expected_skills_all)

    # Find the common elements between list1 and list2
    common_elements_all = set(expected_skills_all).intersection(set(generated_skills_all))

    # Find the number of common elements, considering duplicates
    num_common_elements_all = sum(min(count_gen_all[element], count_exp_all[element]) for element in common_elements_all)



    # Calculate the percentage of matching skills
    if len(expected_skills_all) > 0:
        match_score_all = len(expected_skills_all.intersection(generated_skills_all)) / len(expected_skills_all) * 100
    else:
        match_score_all = 0.0
    # Calculate the percentage of matching skills
    if len(expected_skills_C) > 0:
        match_score_C = len(expected_skills_C.intersection(generated_skills_C)) / len(expected_skills_C) * 100
    else:
        match_score_C = 0.0
    
    # Assign the match score to the 'skills_match_score' column
    results_chatGPT_df.at[index, 'percent_skills_match_all'] = match_score_all
    results_chatGPT_df.at[index, 'percent_skills_match_C'] = match_score_C
    results_chatGPT_df.at[index, 'num_skills_match_all'] = num_common_elements_all
    results_chatGPT_df.at[index, 'num_skills_match_C'] = num_common_elements_C

In [13]:
results_chatGPT_df[results_chatGPT_df['percent_skills_match_all'] == 100]['Problem type'].value_counts()

Problem type
OWA     12
HOWA     7
MC       4
NONE     2
MWA      2
MDC      1
Name: count, dtype: int64

In [14]:
results_chatGPT_df[results_chatGPT_df['percent_skills_match_all'] == 100]['Problem type'].value_counts()

Problem type
OWA     12
HOWA     7
MC       4
NONE     2
MWA      2
MDC      1
Name: count, dtype: int64

In [15]:
import Levenshtein
import re 
def remove_sp_char(sent):
    modified_text = re.sub(r'(<)?(?:\/?(?:SoftSkill(?:_C)?))?(>)', '', sent)
    modified_text = re.sub(r'\n', ' ', modified_text)
    modified_text = re.sub(r'\r', ' ', modified_text)
    return modified_text

results_chatGPT_df['cleaned_generated_text'] = results_chatGPT_df['generated_text'].apply(remove_sp_char)
results_chatGPT_df['cleaned_expected_text'] = results_chatGPT_df['expected_text'].apply(remove_sp_char)
results_chatGPT_df['sentence_levenshtein'] = results_chatGPT_df.apply(lambda row: Levenshtein.distance(row['cleaned_generated_text'], row['cleaned_expected_text']), axis=1)
# results_chatGPT_df['skills_levenshtein'] = results_chatGPT_df.apply(lambda row: Levenshtein.distance(str(row['generated_skills']), str(row['expected_skills'])), axis=1)

In [16]:
# Define functions to calculate Rouge and BLEU scores
def calculate_rouge_scores(expected_list, generated_list):
    rouge = Rouge()
    rouge_scores = []
    min_length = min(len(expected_list), len(generated_list))
    for i in range(min_length):
        expected = expected_list[i]
        generated = generated_list[i]
        scores = rouge.get_scores(expected, generated)
        rouge_scores.append(scores)
    return rouge_scores

def calculate_bleu_score(expected_list, generated_list):
    smoothing = SmoothingFunction().method0
    bleu_scores = []
    for expected, generated in zip(expected_list, generated_list):
        if len(expected_list) == len(generated_list):
            bleu_score = sentence_bleu([expected], generated, smoothing_function=smoothing)
            bleu_scores.append(bleu_score)
    return bleu_scores

def calculate_levenshtein_distance(expected_list, generated_list):
    levenshtein_distances = []
    for expected, generated in zip(expected_list, generated_list):
        if len(expected_list) == len(generated_list):
            distance = Levenshtein.distance(expected, generated)
            levenshtein_distances.append(distance)
    return levenshtein_distances

# Calculate the Rouge and BLEU scores for each row and store them in new columns
results_chatGPT_df['rouge_scores_skill_C'] = results_chatGPT_df.apply(lambda row: calculate_rouge_scores(row['expected_skills_C'], row['generated_skills_C']), axis=1)
results_chatGPT_df['bleu_scores_skill_C'] = results_chatGPT_df.apply(lambda row: calculate_bleu_score(row['expected_skills_C'], row['generated_skills_C']), axis=1)

# Calculate the average scores for each pair individually
results_chatGPT_df['rouge1_avg_skill_C'] = results_chatGPT_df['rouge_scores_skill_C'].apply(lambda x: sum(score[0]['rouge-1']['f'] for score in x) / len(x) if len(x) > 0 else-1)
results_chatGPT_df['rouge2_avg_skill_C'] = results_chatGPT_df['rouge_scores_skill_C'].apply(lambda x: sum(score[0]['rouge-2']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_chatGPT_df['rougeL_avg_skill_C'] = results_chatGPT_df['rouge_scores_skill_C'].apply(lambda x: sum(score[0]['rouge-l']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_chatGPT_df['bleu_avg_skill_C'] = results_chatGPT_df['bleu_scores_skill_C'].apply(lambda x: sum(x) / len(x) if len(x) > 0 else -1)

# Calculate the Levenshtein distances for each pair individually and store the sum in a new column
results_chatGPT_df['levenshtein_skill_C'] = results_chatGPT_df.apply(lambda row: sum(calculate_levenshtein_distance(row['expected_skills_C'], row['generated_skills_C'])), axis=1)


# Calculate the Rouge and BLEU scores for each row and store them in new columns
results_chatGPT_df['rouge_scores_skill_all'] = results_chatGPT_df.apply(lambda row: calculate_rouge_scores(row['expected_skills_all'], row['generated_skills_all']), axis=1)
results_chatGPT_df['bleu_scores_skill_all'] = results_chatGPT_df.apply(lambda row: calculate_bleu_score(row['expected_skills_all'], row['generated_skills_all']), axis=1)

# Calculate the average scores for each pair individually
results_chatGPT_df['rouge1_avg_skill_all'] = results_chatGPT_df['rouge_scores_skill_all'].apply(lambda x: sum(score[0]['rouge-1']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_chatGPT_df['rouge2_avg_skill_all'] = results_chatGPT_df['rouge_scores_skill_all'].apply(lambda x: sum(score[0]['rouge-2']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_chatGPT_df['rougeL_avg_skill_all'] = results_chatGPT_df['rouge_scores_skill_all'].apply(lambda x: sum(score[0]['rouge-l']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_chatGPT_df['bleu_avg_skill_all'] = results_chatGPT_df['bleu_scores_skill_all'].apply(lambda x: sum(x) / len(x) if len(x) > 0 else -1)

# Calculate the Levenshtein distances for each pair individually and store the sum in a new column
results_chatGPT_df['levenshtein_skill_all'] = results_chatGPT_df.apply(lambda row: sum(calculate_levenshtein_distance(row['expected_skills_all'], row['generated_skills_all'])), axis=1)

# results_chatGPT_df[results_chatGPT_df['orig_idx']==694]



In [17]:
results_chatGPT_df.columns

Index(['input', 'generated_text', 'expected_text', 'Wrong chatGPT',
       'Problem type', 'expected_skills_all', 'generated_skills_all',
       'input_skills_C', 'input_skills_all', 'expected_skills_C',
       'generated_skills_C', 'num_skills_all', 'num_skills_C',
       'num_gen_skills_all', 'num_gen_skills_C', 'percent_skills_match_all',
       'percent_skills_match_C', 'num_skills_match_all', 'num_skills_match_C',
       'cleaned_generated_text', 'cleaned_expected_text',
       'sentence_levenshtein', 'rouge_scores_skill_C', 'bleu_scores_skill_C',
       'rouge1_avg_skill_C', 'rouge2_avg_skill_C', 'rougeL_avg_skill_C',
       'bleu_avg_skill_C', 'levenshtein_skill_C', 'rouge_scores_skill_all',
       'bleu_scores_skill_all', 'rouge1_avg_skill_all', 'rouge2_avg_skill_all',
       'rougeL_avg_skill_all', 'bleu_avg_skill_all', 'levenshtein_skill_all'],
      dtype='object')

In [265]:
# results_chatGPT_df.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/bloom-7b1-lora-ORIG-stopseq20230717114608.csv")


In [18]:
results_chatGPT_df['Problem type'].value_counts()

Problem type
OWA     15
HOWA     8
MC       5
MWA      4
NONE     2
MDC      1
Name: count, dtype: int64

In [19]:
pd.set_option('display.max_columns', None)  # Display all columns
# results_chatGPT_df[results_chatGPT_df['num_skills_C'] != results_chatGPT_df['num_gen_skills_C']]
results_chatGPT_df[results_chatGPT_df['num_skills_all'] != results_chatGPT_df['num_gen_skills_all']]
results_chatGPT_df['mismatched_ALL'] = results_chatGPT_df['num_skills_all'] != results_chatGPT_df['num_gen_skills_all']
results_chatGPT_df[results_chatGPT_df['num_skills_all'] != results_chatGPT_df['num_gen_skills_all']].head(3)

,input,generated_text,expected_text,Wrong chatGPT,Problem type,expected_skills_all,generated_skills_all,input_skills_C,input_skills_all,expected_skills_C,generated_skills_C,num_skills_all,num_skills_C,num_gen_skills_all,num_gen_skills_C,percent_skills_match_all,percent_skills_match_C,num_skills_match_all,num_skills_match_C,cleaned_generated_text,cleaned_expected_text,sentence_levenshtein,rouge_scores_skill_C,bleu_scores_skill_C,rouge1_avg_skill_C,rouge2_avg_skill_C,rougeL_avg_skill_C,bleu_avg_skill_C,levenshtein_skill_C,rouge_scores_skill_all,bleu_scores_skill_all,rouge1_avg_skill_all,rouge2_avg_skill_all,rougeL_avg_skill_all,bleu_avg_skill_all,levenshtein_skill_all,mismatched_ALL
4,"Für unsere Abteilung Pâtisserie\nin der Deliciel in Birmenstorf\nsuchen wir\nper sofort\noder nach\nVereinbarung eine <SoftSkill_C>selbständige</SoftSkill_C>, <SoftSkill>motivierte Persönlichkeit</SoftSkill> als\nBäcke\nr",Für unsere Abteilung Pâtisserie\nin der Deliciel in Birmenstorf\nsuchen wir\nper sofort\noder nach\nVereinbarung eine <SoftSkill>selbständige und motivierte Persönlichkeit</SoftSkill> als Bäcker.,Für unsere Abteilung Pâtisserie\nin der Deliciel in Birmenstorf\nsuchen wir\nper sofort\noder nach\nVereinbarung eine <SoftSkill>selbständige Persönlichkeit</SoftSkill>und <SoftSkill>motivierte Persönlichkeit</SoftSkill> als Bäcker.,True,OWA,"[selbständige Persönlichkeit, motivierte Persönlichkeit]",[selbständige und motivierte Persönlichkeit],[selbständige],"[selbständige, motivierte Persönlichkeit]",[selbständige Persönlichkeit],[],2,1,1,0,0.0,0.0,0.0,0.0,Für unsere Abteilung Pâtisserie in der Deliciel in Birmenstorf suchen wir per sofort oder nach Vereinbarung eine selbständige und motivierte Persönlichkeit als Bäcker.,Für unsere Abteilung Pâtisserie in der Deliciel in Birmenstorf suchen wir per sofort oder nach Vereinbarung eine selbständige Persönlichkeitund motivierte Persönlichkeit als Bäcker.,14,[],[],-1.0,-1.0,-1.0,-1.0,0,"[[{'rouge-1': {'r': 0.5, 'p': 1.0, 'f': 0.6666666622222223}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.5, 'p': 1.0, 'f': 0.6666666622222223}}]]",[],0.666667,0.0,0.666667,-1.0,0,True


In [22]:
results_C = results_chatGPT_df[['input', 'expected_text', 'generated_text', 'input_skills_C', 'expected_skills_C', 'generated_skills_C',
                'num_skills_C', 'num_gen_skills_C', 'percent_skills_match_C', 'num_skills_match_C',
                'rouge_scores_skill_C', 'bleu_scores_skill_C', 'rouge1_avg_skill_C',
                'rouge2_avg_skill_C', 'rougeL_avg_skill_C', 'bleu_avg_skill_C', 'levenshtein_skill_C', 'mismatched_ALL','Problem type']]

# Create "results_ALL" DataFrame
results_ALL = results_chatGPT_df[['input', 'expected_text', 'generated_text', 'input_skills_all', 'expected_skills_all', 'generated_skills_all',
                  'num_skills_all', 'num_gen_skills_all', 'percent_skills_match_all', 'num_skills_match_all',
                  'rouge_scores_skill_all', 'bleu_scores_skill_all', 'rouge1_avg_skill_all',
                  'rouge2_avg_skill_all', 'rougeL_avg_skill_all', 'bleu_avg_skill_all', 'levenshtein_skill_all', 'mismatched_ALL','Problem type']]

# Create "results_SENTENCE" DataFrame
results_SENTENCE = results_chatGPT_df[['input', 'expected_text', 'generated_text', 'cleaned_generated_text', 'cleaned_expected_text', 'sentence_levenshtein','Problem type']]

# # Export the three DataFrames to separate CSV files (optional)
results_C.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/GPT3_SS/gpt_part_3_results_C.csv", index=False)
results_ALL.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/GPT3_SS/gpt_part_3_results_ALL.csv", index=False)
results_SENTENCE.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/GPT3_SS/gpt_part_3_results_SENTENCE.csv", index=False)

In [23]:
results_ALL[results_ALL['mismatched_ALL']==True].head(5)

,input,expected_text,generated_text,input_skills_all,expected_skills_all,generated_skills_all,num_skills_all,num_gen_skills_all,percent_skills_match_all,num_skills_match_all,rouge_scores_skill_all,bleu_scores_skill_all,rouge1_avg_skill_all,rouge2_avg_skill_all,rougeL_avg_skill_all,bleu_avg_skill_all,levenshtein_skill_all,mismatched_ALL,Problem type
4,"Für unsere Abteilung Pâtisserie\nin der Deliciel in Birmenstorf\nsuchen wir\nper sofort\noder nach\nVereinbarung eine <SoftSkill_C>selbständige</SoftSkill_C>, <SoftSkill>motivierte Persönlichkeit</SoftSkill> als\nBäcke\nr",Für unsere Abteilung Pâtisserie\nin der Deliciel in Birmenstorf\nsuchen wir\nper sofort\noder nach\nVereinbarung eine <SoftSkill>selbständige Persönlichkeit</SoftSkill>und <SoftSkill>motivierte Persönlichkeit</SoftSkill> als Bäcker.,Für unsere Abteilung Pâtisserie\nin der Deliciel in Birmenstorf\nsuchen wir\nper sofort\noder nach\nVereinbarung eine <SoftSkill>selbständige und motivierte Persönlichkeit</SoftSkill> als Bäcker.,"[selbständige, motivierte Persönlichkeit]","[selbständige Persönlichkeit, motivierte Persönlichkeit]",[selbständige und motivierte Persönlichkeit],2,1,0.0,0.0,"[[{'rouge-1': {'r': 0.5, 'p': 1.0, 'f': 0.6666666622222223}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.5, 'p': 1.0, 'f': 0.6666666622222223}}]]",[],0.666667,0.0,0.666667,-1.0,0,True,OWA


In [24]:
pd.set_option('display.max_row', None)
results_C['levenshtein_skill_C'].mean()

3.5714285714285716

## Sentence ROUGE

In [25]:
from rouge import Rouge

# Function to calculate Rouge scores
def calculate_rouge_scores(reference, candidate):
    rouge = Rouge()
    scores = rouge.get_scores(reference, candidate)
    rouge_1_f = scores[0]['rouge-1']['f'] if 'rouge-1' in scores[0] else None
    rouge_2_f = scores[0]['rouge-2']['f'] if 'rouge-2' in scores[0] else None
    rouge_l_f = scores[0]['rouge-l']['f'] if 'rouge-l' in scores[0] else None
    return rouge_1_f, rouge_2_f, rouge_l_f

# Apply the functions to calculate the scores and add them as new columns
results_SENTENCE['rouge1_score'], results_SENTENCE['rouge2_score'], results_SENTENCE['rougeL_score'] = zip(*results_SENTENCE.apply(
    lambda row: calculate_rouge_scores(row['cleaned_expected_text'], row['cleaned_generated_text']),
    axis=1
))

/tmp/ipykernel_1719172/809110196.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_SENTENCE['rouge1_score'], results_SENTENCE['rouge2_score'], results_SENTENCE['rougeL_score'] = zip(*results_SENTENCE.apply(
/tmp/ipykernel_1719172/809110196.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_SENTENCE['rouge1_score'], results_SENTENCE['rouge2_score'], results_SENTENCE['rougeL_score'] = zip(*results_SENTENCE.apply(
/tmp/ipykernel_1719172/809110196.py:13: SettingWithCopyWarning: 
A value is 

## Sentence BERT Cosine Similarity

In [26]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch

# Assuming you have already read the DataFrame from the CSV file
# results_SENTENCE = pd.read_csv("/path/to/results_SENTENCE.csv")

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to calculate cosine similarity
def calculate_cosine_similarity(sentence1, sentence2):
    # Tokenize and convert sentences to tensors
    inputs = tokenizer([sentence1, sentence2], return_tensors="pt", padding=True, truncation=True)

    # Get BERT embeddings
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :].numpy()

    # Normalize embeddings
    normalized_embeddings = embeddings / (embeddings ** 2).sum(axis=-1, keepdims=True) ** 0.5

    # Calculate cosine similarity
    similarity = (normalized_embeddings[0] @ normalized_embeddings[1].T).item()
    return similarity

# Apply the function to calculate cosine similarity and add it as a new column
results_SENTENCE['cosine_similarity'] = results_SENTENCE.apply(lambda row: calculate_cosine_similarity(row['cleaned_generated_text'], row['cleaned_expected_text']), axis=1)


/tmp/ipykernel_1719172/2919956664.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_SENTENCE['cosine_similarity'] = results_SENTENCE.apply(lambda row: calculate_cosine_similarity(row['cleaned_generated_text'], row['cleaned_expected_text']), axis=1)


## FUnction for cosine similarity for _C and ALL

In [27]:

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to get BERT embeddings for a list of sentences
def get_bert_embeddings(sentences):
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :].numpy()
    return embeddings

# Function to calculate cosine similarity between two lists of sentences
def calculate_cosine_similarity(list1, list2):
    if not list1 or not list2:
        return np.nan

    # Get BERT embeddings for both lists of sentences
    embeddings1 = get_bert_embeddings(list1)
    embeddings2 = get_bert_embeddings(list2)

    # Normalize embeddings
    normalized_embeddings1 = embeddings1 / (embeddings1 ** 2).sum(axis=-1, keepdims=True) ** 0.5
    normalized_embeddings2 = embeddings2 / (embeddings2 ** 2).sum(axis=-1, keepdims=True) ** 0.5

    # Calculate cosine similarity for each sentence pair
    similarity_values = np.dot(normalized_embeddings1, normalized_embeddings2.T)
    return similarity_values.diagonal().tolist()


In [28]:

# Apply the function to calculate pairwise cosine similarity and store the distances in a list
cosine_similarity_list = []
for index, row in results_C.iterrows():
    list1 = row['expected_skills_C']
    list2 = row['generated_skills_C']

    # Calculate cosine similarity for each sentence pair in both lists
    pairwise_similarities = calculate_cosine_similarity(list1, list2)

    # Append the list of pairwise similarities for the row
    cosine_similarity_list.append(pairwise_similarities)

# Add the list of pairwise cosine similarity distances as a new column in the DataFrame
results_C['pairwise_cosine_similarity'] = cosine_similarity_list



# Apply the function to calculate pairwise cosine similarity and store the distances in a list
cosine_similarity_list = []
for index, row in results_ALL.iterrows():
    list1 = row['expected_skills_all']
    list2 = row['generated_skills_all']

    # Calculate cosine similarity for each sentence pair in both lists
    pairwise_similarities = calculate_cosine_similarity(list1, list2)

    # Append the list of pairwise similarities for the row
    cosine_similarity_list.append(pairwise_similarities)

# Add the list of pairwise cosine similarity distances as a new column in the DataFrame
results_ALL['pairwise_cosine_similarity'] = cosine_similarity_list

results_C['average_cosine_similarity'] = results_C['pairwise_cosine_similarity'].apply(lambda x: np.mean(x) if x else np.nan)
results_ALL['average_cosine_similarity'] = results_ALL['pairwise_cosine_similarity'].apply(lambda x: np.mean(x) if x else np.nan)


/tmp/ipykernel_1719172/2476177218.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_C['pairwise_cosine_similarity'] = cosine_similarity_list
/tmp/ipykernel_1719172/2476177218.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_ALL['pairwise_cosine_similarity'] = cosine_similarity_list
/tmp/ipykernel_1719172/2476177218.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [275]:
# results_ALL[results_ALL['mismatched_ALL']==True].head(5)

## Prepare Metrics and Tables 

In [29]:
# Create a DataFrame for _C metrics
summary_C_df = pd.DataFrame({
    'Metric': ['Average Rouge-1 Score (C)', 'Average Rouge-2 Score (C)',
               'Average Rouge-L Score (C)',  'Average Levenshtein (C)',
               'Weighted Avg. Percentage Skills (C)', 'Cosine Similarity(C)'],
    'Value': [results_C['rouge1_avg_skill_C'].mean(), results_C['rouge2_avg_skill_C'].mean(),
              results_C['rougeL_avg_skill_C'].mean(),results_C['levenshtein_skill_C'].mean(),
              results_C['num_skills_match_C'].sum() / results_C['num_skills_C'].sum(),results_C['average_cosine_similarity'].mean()]
})

# Create a DataFrame for "ALL" metrics
summary_ALL_df = pd.DataFrame({
    'Metric': ['Average Rouge-1 Score (ALL)', 'Average Rouge-2 Score (ALL)',
               'Average Rouge-L Score (ALL)', 'Average Levenshtein (ALL)',
               'Weighted Avg. Percentage Skills (ALL)', 'Cosine Similarity(ALL)'],
    'Value': [results_ALL['rouge1_avg_skill_all'].mean(), results_ALL['rouge2_avg_skill_all'].mean(),
              results_ALL['rougeL_avg_skill_all'].mean(), results_ALL['levenshtein_skill_all'].mean(),
              results_ALL['num_skills_match_all'].sum() / results_ALL['num_skills_all'].sum(),results_ALL['average_cosine_similarity'].mean()]
})

# Create a DataFrame for "ALL" metrics
summary_SENTENCE_df = pd.DataFrame({
    'Metric': ['Average Rouge-1 Score (SENTENCE)', 'Average Rouge-2 Score (SENTENCE)',
               'Average Rouge-L Score (SENTENCE)', 'Average Levenshtein (SENTENCE)',
               'Cosine Similarity(SENTENCE)'],
    'Value': [results_SENTENCE['rouge1_score'].mean(), results_SENTENCE['rouge2_score'].mean(),
              results_SENTENCE['rougeL_score'].mean(), results_SENTENCE['sentence_levenshtein'].mean(),
              results_SENTENCE['cosine_similarity'].mean()]
})

# Export the summary DataFrames to CSV
# summary_C_df.to_csv("summary_metrics_C.csv", index=False)
# summary_ALL_df.to_csv("summary_metrics_ALL.csv", index=False)

In [30]:
summary_C_df

,Metric,Value
0,Average Rouge-1 Score (C),0.882449
1,Average Rouge-2 Score (C),0.778571
2,Average Rouge-L Score (C),0.882449
3,Average Levenshtein (C),3.571429
4,Weighted Avg. Percentage Skills (C),0.860465
5,Cosine Similarity(C),0.982694


In [31]:
summary_ALL_df

,Metric,Value
0,Average Rouge-1 Score (ALL),0.953367
1,Average Rouge-2 Score (ALL),0.759456
2,Average Rouge-L Score (ALL),0.953367
3,Average Levenshtein (ALL),4.628571
4,Weighted Avg. Percentage Skills (ALL),0.915966
5,Cosine Similarity(ALL),0.990712


In [32]:
summary_SENTENCE_df

,Metric,Value
0,Average Rouge-1 Score (SENTENCE),0.996104
1,Average Rouge-2 Score (SENTENCE),0.961048
2,Average Rouge-L Score (SENTENCE),0.996104
3,Average Levenshtein (SENTENCE),4.942857
4,Cosine Similarity(SENTENCE),0.995598


In [279]:
summary_SENTENCE_df

,Metric,Value
0,Average Rouge-1 Score (SENTENCE),0.995703
1,Average Rouge-2 Score (SENTENCE),0.985159
2,Average Rouge-L Score (SENTENCE),0.995703
3,Average Levenshtein (SENTENCE),1.756849
4,Cosine Similarity(SENTENCE),0.998346


## Problem Types DATA RESULTS _C SKILLS

In [34]:
summary_C_df_list = []

# Group the DataFrame by "Problem Type"
grouped_results_C = results_C.groupby('Problem type')

# Iterate over each group and calculate the summary DataFrame
for problem_type, group in grouped_results_C:
    num_samples = len(group)  # Calculate the number of samples for the current category
    summary_C_df = pd.DataFrame({
        'Average Rouge-1 Score (C)': [group['rouge1_avg_skill_C'].mean()],
        'Average Rouge-2 Score (C)': [group['rouge2_avg_skill_C'].mean()],
        'Average Rouge-L Score (C)': [group['rougeL_avg_skill_C'].mean()],
        'Average Levenshtein (C)': [group['levenshtein_skill_C'].mean()],
        'Weighted Avg. Percentage Skills (C)': [group['num_skills_match_C'].sum() / group['num_skills_C'].sum()],
        'Cosine Similarity (C)': [group['average_cosine_similarity'].mean()],
        'Number of Samples': [num_samples]
    }, index=[problem_type])
    # Add the summary DataFrame for the current category to the list
    summary_C_df_list.append(summary_C_df)

# Concatenate the summary DataFrames into a single DataFrame
result_summary_df = pd.concat(summary_C_df_list)

# Calculate overall averages across all categories and add as a new row
overall_averages = {
    'Average Rouge-1 Score (C)': results_C['rouge1_avg_skill_C'].mean(),
    'Average Rouge-2 Score (C)': results_C['rouge2_avg_skill_C'].mean(),
    'Average Rouge-L Score (C)': results_C['rougeL_avg_skill_C'].mean(),
    'Average Levenshtein (C)': results_C['levenshtein_skill_C'].mean(),
    'Weighted Avg. Percentage Skills (C)': results_C['num_skills_match_C'].sum() / results_C['num_skills_C'].sum(),
    'Cosine Similarity (C)': results_C['average_cosine_similarity'].mean(),
    'Number of Samples': len(results_C)
}

overall_summary_df = pd.DataFrame(overall_averages, index=['All Samples'])

# Concatenate the overall summary as a new row to the result_summary_df
result_summary_df_C_skills = pd.concat([result_summary_df, overall_summary_df])

result_summary_df_C_skills['model'] = 'chatGPT'
result_summary_df_C_skills


,Average Rouge-1 Score (C),Average Rouge-2 Score (C),Average Rouge-L Score (C),Average Levenshtein (C),Weighted Avg. Percentage Skills (C),Cosine Similarity (C),Number of Samples,model
HOWA,1.000000,0.875000,1.000000,0.000000,1.000000,1.000000,8,chatGPT
MC,0.906667,0.850000,0.906667,8.600000,0.818182,0.979917,5,chatGPT
MDC,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1,chatGPT
MWA,0.671429,0.500000,0.671429,17.250000,0.600000,0.912172,4,chatGPT
NONE,1.000000,0.500000,1.000000,0.000000,1.000000,1.000000,2,chatGPT
OWA,0.844444,0.800000,0.844444,0.866667,0.866667,0.990236,15,chatGPT
All Samples,0.882449,0.778571,0.882449,3.571429,0.860465,0.982694,35,chatGPT


In [35]:
result_summary_df_C_skills['Problem Type'] = result_summary_df_C_skills.index
new_df = result_summary_df_C_skills.iloc[:, [2, 3, 4, 5, 8]]
latex_table = new_df.to_latex(index=False)
print(latex_table)

\begin{tabular}{rrrrl}
\toprule
Average Rouge-L Score (C) & Average Levenshtein (C) & Weighted Avg. Percentage Skills (C) & Cosine Similarity (C) & Problem Type \\
\midrule
1.000000 & 0.000000 & 1.000000 & 1.000000 & HOWA \\
0.906667 & 8.600000 & 0.818182 & 0.979917 & MC \\
1.000000 & 0.000000 & 1.000000 & 1.000000 & MDC \\
0.671429 & 17.250000 & 0.600000 & 0.912172 & MWA \\
1.000000 & 0.000000 & 1.000000 & 1.000000 & NONE \\
0.844444 & 0.866667 & 0.866667 & 0.990236 & OWA \\
0.882449 & 3.571429 & 0.860465 & 0.982694 & All Samples \\
\bottomrule
\end{tabular}



## Problem Types DATA RESULTS ALL SKILLS

In [36]:
# Create an empty list to store the summary DataFrames for each category
summary_ALL_df_list = []

# Group the DataFrame by "Problem Type"
grouped_results_ALL = results_ALL.groupby('Problem type')

# Iterate over each group and calculate the summary DataFrame
for problem_type, group in grouped_results_ALL:
    num_samples = len(group)  # Calculate the number of samples for the current category
    summary_ALL_df = pd.DataFrame({
        'Average Rouge-1 Score (ALL)': [group['rouge1_avg_skill_all'].mean()],
        'Average Rouge-2 Score (ALL)': [group['rouge2_avg_skill_all'].mean()],
        'Average Rouge-L Score (ALL)': [group['rougeL_avg_skill_all'].mean()],
        'Average Levenshtein (ALL)': [group['levenshtein_skill_all'].mean()],
        'Weighted Avg. Percentage Skills (ALL)': [group['num_skills_match_all'].sum() / group['num_skills_all'].sum()],
        'Cosine Similarity (ALL)': [group['average_cosine_similarity'].mean()],
        'Number of Samples': [num_samples]
    }, index=[problem_type])
    # Add the summary DataFrame for the current category to the list
    summary_ALL_df_list.append(summary_ALL_df)

# Concatenate the summary DataFrames into a single DataFrame
result_summary_ALL_df = pd.concat(summary_ALL_df_list)

# Calculate overall averages across all categories and append as a new row
overall_averages_ALL = {
    'Average Rouge-1 Score (ALL)': results_ALL['rouge1_avg_skill_all'].mean(),
    'Average Rouge-2 Score (ALL)': results_ALL['rouge2_avg_skill_all'].mean(),
    'Average Rouge-L Score (ALL)': results_ALL['rougeL_avg_skill_all'].mean(),
    'Average Levenshtein (ALL)': results_ALL['levenshtein_skill_all'].mean(),
    'Weighted Avg. Percentage Skills (ALL)': results_ALL['num_skills_match_all'].sum() / results_ALL['num_skills_all'].sum(),
    'Cosine Similarity (ALL)': results_ALL['average_cosine_similarity'].mean(),
    'Number of Samples': len(results_ALL)
}

overall_summary_ALL_df = pd.DataFrame(overall_averages_ALL, index=['All Samples'])

# Concatenate the overall summary as a new row to the result_summary_ALL_df
result_summary_df_ALL_skills = pd.concat([result_summary_ALL_df, overall_summary_ALL_df])

result_summary_df_ALL_skills['model'] = 'chatGPT'
result_summary_df_ALL_skills


,Average Rouge-1 Score (ALL),Average Rouge-2 Score (ALL),Average Rouge-L Score (ALL),Average Levenshtein (ALL),Weighted Avg. Percentage Skills (ALL),Cosine Similarity (ALL),Number of Samples,model
HOWA,0.979167,0.528869,0.979167,1.250000,0.968750,0.993579,8,chatGPT
MC,0.953333,0.875000,0.953333,8.600000,0.900000,0.989959,5,chatGPT
MDC,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1,chatGPT
MWA,0.810714,0.608333,0.810714,22.750000,0.750000,0.955328,4,chatGPT
NONE,1.000000,0.666667,1.000000,0.000000,1.000000,1.000000,2,chatGPT
OWA,0.968333,0.880556,0.968333,1.200000,0.911111,0.997012,15,chatGPT
All Samples,0.953367,0.759456,0.953367,4.628571,0.915966,0.990712,35,chatGPT


In [37]:
result_summary_df_ALL_skills['Problem Type'] = result_summary_df_ALL_skills.index
new_df = result_summary_df_ALL_skills.iloc[:, [2, 3, 4, 5, 8]]
latex_table = new_df.to_latex(index=False)
print(latex_table)

\begin{tabular}{rrrrl}
\toprule
Average Rouge-L Score (ALL) & Average Levenshtein (ALL) & Weighted Avg. Percentage Skills (ALL) & Cosine Similarity (ALL) & Problem Type \\
\midrule
0.979167 & 1.250000 & 0.968750 & 0.993579 & HOWA \\
0.953333 & 8.600000 & 0.900000 & 0.989959 & MC \\
1.000000 & 0.000000 & 1.000000 & 1.000000 & MDC \\
0.810714 & 22.750000 & 0.750000 & 0.955328 & MWA \\
1.000000 & 0.000000 & 1.000000 & 1.000000 & NONE \\
0.968333 & 1.200000 & 0.911111 & 0.997012 & OWA \\
0.953367 & 4.628571 & 0.915966 & 0.990712 & All Samples \\
\bottomrule
\end{tabular}



## Problem Types DATA RESULTS SENTENCE SKILLS

In [38]:
# Create an empty list to store the summary DataFrames for each category
summary_SENTENCE_df_list = []

# Group the DataFrame by "Problem Type"
grouped_results_SENTENCE = results_SENTENCE.groupby('Problem type')

# Iterate over each group and calculate the summary DataFrame
for problem_type, group in grouped_results_SENTENCE:
    num_samples = len(group)  # Calculate the number of samples for the current category
    summary_SENTENCE_df = pd.DataFrame({
        'Average Rouge-1 Score (SENTENCE)': [group['rouge1_score'].mean()],
        'Average Rouge-2 Score (SENTENCE)': [group['rouge2_score'].mean()],
        'Average Rouge-L Score (SENTENCE)': [group['rougeL_score'].mean()],
        'Average Levenshtein (SENTENCE)': [group['sentence_levenshtein'].mean()],
        'Cosine Similarity (SENTENCE)': [group['cosine_similarity'].mean()],
        'Number of Samples': [num_samples]
    }, index=[problem_type])
    # Add the summary DataFrame for the current category to the list
    summary_SENTENCE_df_list.append(summary_SENTENCE_df)

# Concatenate the summary DataFrames into a single DataFrame
result_summary_SENTENCE_df = pd.concat(summary_SENTENCE_df_list)

# Calculate overall averages across all categories and add as a new row
overall_averages_SENTENCE = {
    'Average Rouge-1 Score (SENTENCE)': results_SENTENCE['rouge1_score'].mean(),
    'Average Rouge-2 Score (SENTENCE)': results_SENTENCE['rouge2_score'].mean(),
    'Average Rouge-L Score (SENTENCE)': results_SENTENCE['rougeL_score'].mean(),
    'Average Levenshtein (SENTENCE)': results_SENTENCE['sentence_levenshtein'].mean(),
    'Cosine Similarity (SENTENCE)': results_SENTENCE['cosine_similarity'].mean(),
    'Number of Samples': len(results_SENTENCE)
}

overall_summary_SENTENCE_df = pd.DataFrame(overall_averages_SENTENCE, index=['All Samples'])

# Concatenate the overall summary as a new row to the result_summary_SENTENCE_df
result_summary_df_SENTENCE = pd.concat([result_summary_SENTENCE_df, overall_summary_SENTENCE_df])

result_summary_df_SENTENCE['model'] = 'chatGPT'
result_summary_df_SENTENCE

,Average Rouge-1 Score (SENTENCE),Average Rouge-2 Score (SENTENCE),Average Rouge-L Score (SENTENCE),Average Levenshtein (SENTENCE),Cosine Similarity (SENTENCE),Number of Samples,model
HOWA,1.000000,0.986111,1.000000,1.250000,0.998820,8,chatGPT
MC,1.000000,0.965517,1.000000,8.000000,0.995281,5,chatGPT
MDC,1.000000,1.000000,1.000000,0.000000,1.000000,1,chatGPT
MWA,1.000000,0.848739,1.000000,22.750000,0.972750,4,chatGPT
NONE,1.000000,1.000000,1.000000,0.000000,1.000000,2,chatGPT
OWA,0.990909,0.968350,0.990909,2.133333,0.999199,15,chatGPT
All Samples,0.996104,0.961048,0.996104,4.942857,0.995598,35,chatGPT


In [39]:
result_summary_df_SENTENCE['Problem Type'] = result_summary_df_SENTENCE.index
new_df = result_summary_df_SENTENCE.iloc[:, [2, 3, 4, 7]]
latex_table = new_df.to_latex(index=False)
print(latex_table)

\begin{tabular}{rrrl}
\toprule
Average Rouge-L Score (SENTENCE) & Average Levenshtein (SENTENCE) & Cosine Similarity (SENTENCE) & Problem Type \\
\midrule
1.000000 & 1.250000 & 0.998820 & HOWA \\
1.000000 & 8.000000 & 0.995281 & MC \\
1.000000 & 0.000000 & 1.000000 & MDC \\
1.000000 & 22.750000 & 0.972750 & MWA \\
1.000000 & 0.000000 & 1.000000 & NONE \\
0.990909 & 2.133333 & 0.999199 & OWA \\
0.996104 & 4.942857 & 0.995598 & All Samples \\
\bottomrule
\end{tabular}



In [286]:
# result_summary_df_C_skills.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/GPT3_SS/summary_metrics_C_GPT3-part-2.csv", index=False)
# result_summary_df_ALL_skills.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/GPT3_SS/summary_metrics_ALL_GPT3-part-2.csv", index=False)
# result_summary_df_SENTENCE.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/GPT3_SS/summary_metrics_SENTENCE_GPT3-part-2.csv", index=False)